Image review

In [7]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output

image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/cusp_images"
mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/masks"
out_img_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/overlapping/images"
out_mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/overlapping/masks"

os.makedirs(out_img_dir, exist_ok=True)
os.makedirs(out_mask_dir, exist_ok=True)

# Get sorted list of filenames
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith((".png", ".jpg"))])
index = 0  # start from beginning or last saved index
saved = 0

def show_image():
    global index
    if index >= len(image_files):
        print("✅ Done reviewing all images!")
        return
    
    image_name = image_files[index]
    mask_name = image_name.replace(".jpg", ".png")

    image_path = os.path.join(image_dir, image_name)
    mask_path = os.path.join(mask_dir, mask_name)

    img = np.array(Image.open(image_path).convert("RGB"))
    mask = np.array(Image.open(mask_path).convert("L"))

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img)
    axs[0].set_title("Image")
    axs[0].axis('off')

    axs[1].imshow(mask, cmap='gray')
    axs[1].set_title("Mask")
    axs[1].axis('off')

    plt.suptitle(f"{image_name}")
    plt.show()

def mark_overlapping(_):
    global index, saved
    image_name = image_files[index]
    mask_name = image_name.replace(".jpg", ".png")

    # Copy to overlapping folder
    shutil.copy(os.path.join(image_dir, image_name), os.path.join(out_img_dir, f"overlap_{saved:04}.png"))
    shutil.copy(os.path.join(mask_dir, mask_name), os.path.join(out_mask_dir, f"overlap_{saved:04}.png"))

    saved += 1
    index += 1
    update_display()

def skip(_):
    global index
    index += 1
    update_display()

def update_display():
    clear_output(wait=True)
    show_image()
    display(buttons_box)

# Buttons
btn_overlap = widgets.Button(description="Mark as Overlapping", button_style='danger')
btn_skip = widgets.Button(description="Skip", button_style='success')
btn_overlap.on_click(mark_overlapping)
btn_skip.on_click(skip)

buttons_box = widgets.HBox([btn_overlap, btn_skip])

# Start
update_display()


✅ Done reviewing all images!


Patch creator

In [32]:
import os
import numpy as np
from PIL import Image
import cv2

# Settings
image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/cusp_images"
mask_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/masks"
output_image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/patches128/images"
output_mask_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/patches128/masks"
# overlap_names = set(os.listdir("/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/overlapping/images"))
# overlap_names = {f.replace("overlap_", "").replace(".png", ".jpg") for f in overlap_names}



os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

def extract_cusp_patches(image_name, image_id_start=0, expand_ratio=1.8):
    image_path = os.path.join(image_dir, image_name)
    mask_path = os.path.join(mask_dir, image_name.replace(".jpg", ".png"))

    image = np.array(Image.open(image_path).convert("RGB"))
    mask = np.array(Image.open(mask_path).convert("L"))
    binary_mask = (mask > 0).astype(np.uint8)

    # Connected component analysis
    num_labels, labels = cv2.connectedComponents(binary_mask)
    patch_id = image_id_start

    for i in range(1, num_labels):  # skip background
        component_mask = (labels == i).astype(np.uint8)
        y_idxs, x_idxs = np.nonzero(component_mask)

        if len(x_idxs) == 0 or len(y_idxs) == 0:
            continue

        # Compute bounding box around component
        x_min, x_max = np.min(x_idxs), np.max(x_idxs)
        y_min, y_max = np.min(y_idxs), np.max(y_idxs)

        # Expand bounding box
        width = x_max - x_min
        height = y_max - y_min
        new_width = int(width * expand_ratio)
        new_height = int(height * expand_ratio)

        x_center = (x_min + x_max) // 2
        y_center = (y_min + y_max) // 2

        # Compute final cropping area
        x_start, x_end = x_center - new_width // 2, x_center + new_width // 2
        y_start, y_end = y_center - new_height // 2, y_center + new_height // 2

        # Compute required padding if out of bounds
        pad_x_before = max(0, -x_start)
        pad_x_after = max(0, x_end - image.shape[1])
        pad_y_before = max(0, -y_start)
        pad_y_after = max(0, y_end - image.shape[0])

        # Apply padding
        image_padded = np.pad(image, 
                              ((pad_y_before, pad_y_after), (pad_x_before, pad_x_after), (0, 0)), 
                              mode='constant', constant_values=0)
        mask_padded = np.pad(component_mask, 
                             ((pad_y_before, pad_y_after), (pad_x_before, pad_x_after)), 
                             mode='constant', constant_values=0)

        # Adjust crop coordinates after padding
        new_x_start = x_start + pad_x_before
        new_x_end = x_end + pad_x_before
        new_y_start = y_start + pad_y_before
        new_y_end = y_end + pad_y_before

        # Extract patches
        patch_img = image_padded[new_y_start:new_y_end, new_x_start:new_x_end]
        patch_mask = mask_padded[new_y_start:new_y_end, new_x_start:new_x_end]

        # Save patches
        img_out_path = os.path.join(output_image_dir, f"image_patch_{patch_id:04}.png")
        mask_out_path = os.path.join(output_mask_dir, f"mask_patch_{patch_id:04}.png")

        Image.fromarray(patch_img).save(img_out_path)
        Image.fromarray((patch_mask * 255).astype(np.uint8)).save(mask_out_path)

        patch_id += 1

    return patch_id

# Run on your dataset
image_files = [f for f in os.listdir(image_dir) if f.endswith((".png", ".jpg", ".tif"))]
next_id = 0
for fname in image_files:
    # if fname in overlap_names:
    #     continue
    next_id = extract_cusp_patches(fname, image_id_start=next_id)

print(f"✅ Finished extracting patches. Total patches: {next_id}")


✅ Finished extracting patches. Total patches: 137


In [2]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

# Custom loss functions
from metrics import log_dice_loss, dice_coef, iou  




In [33]:
# Path to previously trained model
pretrained_model_path = "/Users/rfalcao/FYP/Train/UnetBC50tanh.h5"  # Change to actual path

# Load the model with custom metrics
model = load_model(pretrained_model_path, custom_objects={'dice_coef': dice_coef, 'iou': iou})

# Freeze early encoder layers to retain cusp knowledge
for layer in model.layers[:10]:  # Adjust number of frozen layers if needed
    layer.trainable = False

# Compile with a lower learning rate for gradual fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-6), loss='binary_crossentropy', metrics=['binary_accuracy', dice_coef, iou])




In [34]:
# Directories for new patches
patch_image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/patches128/images"  # Change to actual path
patch_mask_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/patches128/masks"

IMG_SIZE = 224  # Input size for U-Net

# Get all patch filenames
patch_filenames = sorted(os.listdir(patch_image_dir))  
mask_filenames = set(os.listdir(patch_mask_dir))  

# Split patches into train (80%) and validation set (20%)
train_filenames, test_filenames = train_test_split(patch_filenames, test_size=0.1, random_state=42)

print(f"Training Patches: {len(train_filenames)} images")
print(f"Test Patches: {len(test_filenames)} images")


Training Patches: 144 images
Test Patches: 17 images


In [35]:
# Data augmentation settings
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.15,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

def load_and_augment_patches(filenames, image_dir, mask_dir, num_augments_per_image=4):
    images, masks = [], []

    for filename in filenames:
        img_path = os.path.join(image_dir, filename)
        mask_filename= filename.replace("image", "mask")

        mask_path = os.path.join(mask_dir, mask_filename) if mask_filename in mask_filenames else None

        # Load image
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Skipping {filename} (missing image)")
            continue

        img = img.astype(np.float32) / 255.0  # Normalize
        img = np.expand_dims(img, axis=-1)

        # Load mask or create an empty one
        if mask_path and os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = mask.astype(np.float32) / 255.0
            mask = np.expand_dims(mask, axis=-1)
        else:
            print(f" {filename} (missing mask)")
            mask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.float32)

        # Apply augmentation multiple times
        for _ in range(num_augments_per_image):
            seed = np.random.randint(10000)
            aug_img = datagen.random_transform(img, seed=seed)
            aug_mask = datagen.random_transform(mask, seed=seed)

            # Resize after augmentation
            aug_img = cv2.resize(aug_img, (IMG_SIZE, IMG_SIZE))
            aug_mask = cv2.resize(aug_mask, (IMG_SIZE, IMG_SIZE))

            images.append(aug_img)
            masks.append(aug_mask)

    return np.array(images), np.array(masks)

# Load and augment patches
X_train, y_train = load_and_augment_patches(train_filenames, patch_image_dir, patch_mask_dir, num_augments_per_image=4)


print(f"Total Augmented Training Samples: {len(X_train)}")



Total Augmented Training Samples: 576


X_train shape: (576, 224, 224, 1)


In [36]:
################### Check for blank masks ###################

print("Checking dataset for blank masks...")
num_blank_masks = np.sum(np.all(y_train == 0, axis=(1,2)))  # Count fully blank masks
print(f"⚠️ Found {num_blank_masks} blank masks out of {len(y_train)} total.")

if num_blank_masks > 0:
    print(f"❌ Your dataset contains {num_blank_masks} fully blank masks! Consider removing or handling them.")


Checking dataset for blank masks...
⚠️ Found 55 blank masks out of 576 total.
❌ Your dataset contains 55 fully blank masks! Consider removing or handling them.


In [25]:
# Early stopping for efficiency
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# Fine-tuning
history = model.fit(
    X_train, y_train,
    epochs=20,  # Reduce epochs to avoid overfitting
    batch_size=4,  # Smaller batch size due to limited patch dataset
    validation_split=0.125,
    callbacks=[early_stopping],
    verbose=1
)




Epoch 1/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 495s 4s/step - binary_accuracy: 0.6698 - dice_coef: 0.3135 - iou: 0.1079 - loss: 0.7014 - val_binary_accuracy: 0.7838 - val_dice_coef: 0.5781 - val_iou: 0.4629 - val_loss: 0.3758
Epoch 2/20
103/126 ━━━━━━━━━━━━━━━━━━━━ 1:29 4s/step - binary_accuracy: 0.7756 - dice_coef: 0.5993 - iou: 0.4802 - loss: 0.3987

KeyboardInterrupt: 

In [7]:
# Save the fine-tuned model
model.save('BC50Tuned.h5')
print("Fine-tuned model saved successfully!")

Fine-tuned model saved successfully!


In [8]:
def load_test_images(filenames, image_dir, mask_dir):
    images, masks = [], []

    for filename in filenames:
        img_path = os.path.join(image_dir, filename)
        
        # ✅ Convert .jpg filename to .png for mask lookup
        mask_filename= filename.replace("image", "mask")
        mask_path = os.path.join(mask_dir, mask_filename) if mask_filename in mask_filenames else None

        # Load image
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Skipping {filename} (missing image)")
            continue

        # Resize image
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype(np.float32) / 255.0
        img = np.expand_dims(img, axis=-1)

        # Load mask or create an empty one
        if mask_path and os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))
            mask = mask.astype(np.float32) / 255.0
            mask = np.expand_dims(mask, axis=-1)
        else:
            mask = np.zeros((IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

# Load test images (no augmentation)
X_test, y_test = load_test_images(test_filenames, patch_image_dir, patch_mask_dir)

print(f"Final Test Set Size: {len(X_test)} images")

Final Test Set Size: 17 images


In [9]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score

def predict_mask(image, model, target_size=(224, 224)):
    """Resize image, predict mask, and resize mask back to original size."""
    original_size = image.shape[:2]
    
    # Normalize and add channel + batch dimensions
    resized_image = cv2.resize(image, target_size)
    resized_image = np.expand_dims(resized_image, axis=-1)  # Add channel dimension
    resized_image = np.expand_dims(resized_image, axis=0)   # Add batch dimension
    resized_image = resized_image.astype(np.float32) / 255.0
    
    # Predict mask
    predicted_mask = model.predict(resized_image)[0, :, :, 0]
    
    # Resize mask back to original image size
    predicted_mask = cv2.resize(predicted_mask, (original_size[1], original_size[0]))
    
    return predicted_mask

def save_plot(image, true_mask, predicted_mask, save_dir, img_index):
    """Save the plot with original image, ground truth mask, and predicted mask."""
    os.makedirs(save_dir, exist_ok=True)  # Ensure the folder exists

    # Normalize predicted mask to [0,1] for visualization
    predicted_mask = (predicted_mask + 1) / 2

    # Create subplot
    plt.figure(figsize=(18, 6))

    # 🔹 Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title("Original Image")
    plt.axis("off")

    # 🔹 Ground Truth Mask
    plt.subplot(1, 3, 2)
    plt.imshow(true_mask, cmap='gray')
    plt.title("Actual Mask (Ground Truth)")
    plt.axis("off")

    # 🔹 Predicted Mask with Probability Legend
    plt.subplot(1, 3, 3)
    mask_plot = plt.imshow(predicted_mask, cmap='jet')
    plt.colorbar(mask_plot, fraction=0.046, pad=0.04)  # ✅ Add colorbar legend
    plt.title("Predicted Mask (Probability)")
    plt.axis("off")

    # Save the figure
    save_path = os.path.join(save_dir, f"prediction_{img_index}.png")
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()  # Close the plot to free memory

# ✅ Loop over test images
save_directory = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/patches128/plots2"  # 🔹 Change this to your desired save folder

for i in range(len(X_test)):
    img = X_test[i].squeeze()  # Original image
    true_mask = y_test[i].squeeze()  # Ground truth mask

    # Predict mask
    predicted_mask = predict_mask(img, model)

    # Save the plot
    save_plot(img, true_mask, predicted_mask, save_directory, i)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 706ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
